| Universidad de los Andes<br>Departamento de Ingeniería Eléctrica y Electrónica<br>IELE 4922 - Reinforcement Learning<br><br><p style="font-size:30px">Taller 2 - Parte II: Value y Policy Iteration </p> 	|[<img src="images/uniandes_logo.png" width="250"/>](images/uniandes_logo.png)  	|
|-----------------------------------------------------------------------------------	|---	|

## Autores
- Juan Camilo Pico Garrido - 201712801
- Mauricio Ricardo Delgado Quintero - 201712801

Este es el cuaderno *(Jupyter Notebook)* base para desarrollar la segunda parte del taller 2 del curso de *Reinforcement Learning (2021-10)*. En este se busca implementar los métodos de *Value Iteration* y *Policy Iteration* en un *gridworld*. Con este entorno se puede visualizar de forma gráfica e interactiva el comportamiento de estos métodos. 

# El entorno: *Gridworld*
El entorno consiste en una cuadrícula de 3x4, donde cada celda corresponde a una posible ubicación del agente o estado. El agente en cada celda tiene cuatro acciones posibles de movimiento: norte, sur, este y oeste (N, S, E, O). Sin embargo, estas acciones NO son confiables. Con una probabilidad de 0.8 el agente se moverá en la dirección prevista, y con una probabilidad de 0.2, el agente se moverá en una dirección aleatoria contigua. Es decir si se quiere mover al norte (N), las direcciones contiguas a las que se podrá mover con una probabilidad de 0.2 serán al este (E) o al oeste (O). Adicionalmente, aquellas acciones que lleven al agente fuera de la cuadrícula o a posiciones bloqueadas, no tendrán efecto en la posición del agente.

El estado inicial del agente será en la celda (2,0). Si el agente llega a la celda con el diamante, recibirá una recompensa de +1.0 y el juego (*episodio*) terminará. Si el agente llega a la celda con la bomba, recibirá una recompensa de -1.0 y el episodio terminará. Por ende, los estados (0,3) y (1,3) serán considerados *estados terminales*. El agente recibe recompensa 0.0 por llegar a cualquier otra celda. 

<center>
    <img src="images/gridworld.png" alt="centered image" width=300/>
</center>

**Consideraciones importantes:**

* Para la correcta ejecución del código dado mantenga los nombres de las variables propuestos.

* Debe editar y completar unicamente las celdas que comiencen con la instrucción #EDITABLE

* No se preocupe si la ventana de gridworld no responde en algunos momentos, una vez ejecute los métodos para la evaluación (Todos aquellos deppues de las instrucciones para la **visualización**) esta se actualizará y podrá ver de forma interactiva la actualización de los valores deseados.

* Si quiere volver a visualizar el gridworld, debe volver a ejecutar al menos todas las celdas de esa sección.

In [1]:
# Librerias que contienen la dinámica del entorno gridworld
from types import MethodType
from gridworld import GridWorld, pygame
import copy

pygame 2.1.2 (SDL 2.0.18, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


Se crea el objeto *gridworld* con las siguientes característica:
* Tamaño de la cuadrícula: 3 filas y 4 columnas
* La celda bloqueda estará en (1,1)
* La celda con la bomba estará en (1,3)
* La celda con el diamante estará en (0,3)

Una vez creado el objeto, en una ventana emergente de pygame se encuentra la visualización del *gridworld*

In [2]:
gw = GridWorld(rows=3, cols=4, walls=[(1,1)], pits=[(1,3)], goals=[(0,3)], live_reward=0.0)

## Espacio de estados
Para acceder a una lista con los estados del gridworld se puede utilizar la propiedad *states* del objeto gridworld, así: `gw.states`

¿Cuántos estados tiene el problema?

In [3]:
# EDITABLE
len(gw.states)

11

**Resultado esperado:**

```
num_states = 11
```


## Espacio de acciones

Para explorar el espacio de acciones puede hacer uso de la función `gw.get_allowed_actions(state)`, que recibe como parámetro el estado. Este estado se define como una tupla (*x,y*), con la posición donde se encuentra el agente.


¿Cuál es el espacio de acciones en el estado inicial?

In [4]:
# EDITABLE
gw.get_allowed_actions((0,0))

['N', 'S', 'E', 'W']

**Resultado esperado:**
```
allowed_actions = ['N', 'S', 'E', 'W']
```

No obstante, se sabe que las acciones no son confiables ya que tienen un ruido inherente. Esto es, con una probabilidad de 0.8 el agente se mueve en la dirección prevista, y con una probabilidad de 0.2, el agente se mueve en una dirección aleatoria hacia los lados.

Por ejemplo, si la acción deseada es 'N', con probabilidad de 0.8 se moverá al norte. Con probabilidad de 0.1 al este y con probabilidad de 0.1 al oeste.

Verifique esto para la acción 'E'. Utilice los atributos `gw.real_actions[action]` y `gw.action_probabilitites`

In [5]:
# EDITABLE
print(gw.real_actions['E'])
print(gw.action_probabilities)

['E', 'N', 'S']
[0.8, 0.1, 0.1]


**Resultado esperado:**
```
['E', 'N', 'S']
[0.8, 0.1, 0.1]
```

## Aplicación de la acción
Una vez se conoce el estado actual y la acción que va a ejecutar el agente, esta se puede aplicar al entorno. En respuesta, el agente percibe el nuevo estado del entorno, así como una señal de recompensa que indica que tan buena fue se acción. Igualmente, se recibe una señal de *done* que indica si se alcanzó un estado terminal. En este caso, el episodio finaliza.

<center>
    <img src="images/rl_bloques.PNG" alt="centered image" width=350/>
</center>

La recompensa del agente del gridworld se define como:
\begin{equation*}
R(s,a,s') = \begin{cases}
1 &\text{si $s=(0,3)$}\\
-1 &\text{si $s=(1,3)$}\\
0 &\text{d.l.c}
\end{cases}
\end{equation*}


El método `gw.step(state, action, random)` permite aplicar una acción al gridworld. Este método retorna el nuevo estado, la recompensa, la accion ejecutada y la bandera de terminado (done), todo en ese orden. 

¿Qué sucede si el agente está en el estado (2,1) y realiza la acción 'N' ? Imprima los resultados con el parámetro `random = False`.

In [44]:
# EDITABLE
gw.step((2,1), 'W', random = False)

((2, 0), 0.0, 'W', False)

**Resultado esperado**
```
new_state = (2, 1)
reward = 0.0
action = N
done = False
```

Note que el agente, al tratar de moverse en la dirección bloqueada, se queda en su misma posición.

Ahora bien, repita esto con el parámetro `random = True`. En este caso, dada la aleatoriedad del entorno, puede que el resultado obtenido no sea igual al anterior. No obstante, la mayoría de las veces (el 80% para ser exacto) sí lo será. Sientase en la libertad de ejecutar varias veces la celda.

In [41]:
# EDITABLE
gw.step((2,1), 'N', random = True)

((2, 1), 0.0, 'N', False)

Finalmente, se reinicia el entorno.

In [30]:
pygame.quit
gw = None

Cierre la ventana emergente y reinicie el kernel de ejecución.

## Función de Valor, $V$

La política $\pi$ indica, para un entorno determinístico, cuál acción debe ejecutarse en cada estado, con el objetivo de que el agente reciba la mayor utilidad. 

Siguiendo una política $\pi$,  el valor de un estado $V(s)$ en el tiempo $t$ formalmente corresponde a la suma descontada de las recompensas recibidas, si el agente tiene como estado inicial $s$ y se comporta óptimamente en adelante. En palabras simples, indica cuánta utilidad  recibiría el agente si comenzara en $s$ y se comportara bien de ahí en adelante, 

$$V_{t}(s)=\sum_{s'} P(s'|s,a)(R(s,a,s')+\gamma V_{t-1}(s'))$$

Donde $\gamma$ es un factor conocido como *tasa de descuento*. Permite balancear la preferencia de tener recompensas a corto o largo plazo.

Dada un política $\pi(s)$ es posible encontrar el valor de cada estado por medio de programación dinámica, así ($\gamma=0.9$):
<center>
    <img src="images/fvalor.png" alt="centered image" width=650/>
</center>

Se puede encontrar el valor de cada estado del gridworld a partir de una política fija. Suponga que tenemos la siguiente política determinística:

<center>
    <img src="images/gridworld2.png" alt="centered image" width=250/>
</center>

In [1]:
from types import MethodType
from gridworld import GridWorld, pygame
import copy

gw = GridWorld(3, 4, [(1,1)], [(1,3)], [(0,3)], 0.0)

pygame 2.1.2 (SDL 2.0.18, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


Defina la política deterministica propuesta en la figura:

In [6]:
# EDITABLE

state = gw.states
action = ['S', 'E', 'E', 'E', 'S', 'N', 'E', 'E', 'E', 'N', 'W']
# --------------------------------------------

for i, s in enumerate(state):
    gw.policy[s] = action[i] 
    
print(gw.policy)

{(0, 0): 'S', (0, 1): 'E', (0, 2): 'E', (0, 3): 'E', (1, 0): 'S', (1, 2): 'N', (1, 3): 'E', (2, 0): 'E', (2, 1): 'E', (2, 2): 'N', (2, 3): 'W'}


**Resultado esperado**
```
{(0, 0): 'S', (0, 1): 'E', (0, 2): 'E', (0, 3): 'E', (1, 0): 'S', (1, 2): 'N', (1, 3): 'E', (2, 0): 'E', (2, 1): 'E', (2, 2): 'N', (2, 3): 'W'}
```

Ahora se implementa la función de actualización para estimar los valores de los estados, según la política $\pi(s)$, definida anteirormente. 

**Tenga en cuenta que:**
* Se va a asumir que las acciones dadas por la política son determinísticas. Esto es, se aplican al entorno con probabilidad 1
* La regla de actualización para estados terminales es: $V_{t}(s)=\sum_{s'} P(s'|s,\pi(s))R(s,a,s')$
* La regla de actualización para estados no terminales es: $V_{t}(s)=\sum_{s'} P(s'|s,\pi(s))(R(s,a,s')+\gamma V_{t-1}(s'))$
* Para obtener el valor de un estado en t-1 usamos `v(s)=gw.state_values[state] `

In [7]:
def update_values(gw, gamma):
    # Arreglos para los valores
    value = dict.fromkeys(gw.states , 0.0)
    new_values = dict.fromkeys(gw.states , 0.0)
    
    for state in gw.states: 
        # Tomar acción de politica dada
        action = gw.policy[state]
        # Dar un paso
        new_state, reward, _, done = gw.step(state, action, random=False)
        # Actualizar valores
        if(done):
            # actualización para un estado terminal
            new_values[state] = reward
        else:
            # actualización para un estado no terminal
            new_values[state] = reward + gamma*gw.state_values[new_state] #borrar
    
    # Copiar valores
    value = copy.deepcopy(new_values) 
    gw.state_values = copy.deepcopy(new_values) 

Para poner a prueba la función se va a escoger una tasa de descuento de 0.9 y a realizar la estimación de los valores durante un horizonte de 15 iteraciones.

In [8]:
gamma = 0.9
H = 15

**Visualización:**
1. Ejecute la siguiente celda y haga *clic* sobre la ventana del gridworld. 
2. Podra ver como se actualizan los valores de cada estado presionando la tecla **Espacio**. 
3. Una vez completetadas las 10 iteraciones, puede cerrar el gridworld presionando la tecla **Esc**.
4. Si tienes un error en la función 'update_values', despues de corregirlo es necesario que vuelvas a correr las 4 celdas anteriores para poder volver a lanzar la interfaz del gridworld

In [9]:
gw.update_values = MethodType(update_values, gw)
gw.solve_dynamic_programming(gamma=gamma, horizon=H)

Al cabo de 10 iteraciones, el valor de los estados, siguiendo la política $\pi(s)$ son:

In [10]:
for state in gw.states:
    print(f'V{state} = {gw.state_values[state]:.3f}')

V(0, 0) = 0.478
V(0, 1) = 0.810
V(0, 2) = 0.900
V(0, 3) = 1.000
V(1, 0) = 0.531
V(1, 2) = 0.810
V(1, 3) = -1.000
V(2, 0) = 0.590
V(2, 1) = 0.656
V(2, 2) = 0.729
V(2, 3) = 0.656


Nota que los estados cercanos a (0,3) tienen valores cercanos y +1. Esto se debe a que el estado terminal (0,3) tiene un valor igual a la recompensa recibida en este estado y se propaga a los demas estados, de acuerdo a la política seguida. Note que los estados mas lejanos a (0,3) como por ejemplo (0,0), (1,0) y (2,0) tienes valores bajos pero siguen siendo positivos. Esto es porque la política define una secuencia de acciones desde cualquiera de estos tres estados al estado terminal (0,3). Sin embargo, la recompensa recibida presenta un mayor descuento, pues se deben realizar más pasos para llegar hasta (0,3).

* Desde (0,0) hasta (0,3) --> 7 pasos --> $v(s)=\gamma^7*R=0.4783$
* Desde (1,0) hasta (0,3) --> 6 pasos --> $v(s)=\gamma^6*R=0.5314$
* Desde (2,0) hasta (0,3) --> 5 pasos --> $v(s)=\gamma^5*R=0.5905$

 Ahora sí, es momento de implementar los métodos que permiten obtener una política para maximizar las recompensas.

## Value iteration $V^{*}$

Este método permite aproximar los valores óptimos de los estados, de tal manera que se pueda encontrar, al final, una política $\pi^{*}(s)$ que permita maximizar la recompensa recibida por el agente a lo largo del horizonte.

$$V^{*}(s)=\max_{\pi} \mathbb{E}\left[ \sum_{t=0}^{H} \gamma^{t} R(s_t,a_t,s_{t+1}) | \pi, s_0=s \right]$$

Cuando el horizonte es $H=0$, el valor de todos los estado es igual a su valor de inicialización, generalmente cero.

In [53]:
from types import MethodType
from gridworld import GridWorld, pygame
import copy
from colorama import Fore

gw = GridWorld(3, 4, [(1,1)], [(1,3)], [(0,3)], 0.0)

In [54]:
# Inicializar valores de los estados
gw.state_values = gw.init_values()

Verifique los valores de los siguientes estados: (2,0), (0,3), (1,3)

In [55]:
# EDITABLE
print(gw.state_values[(2,0)])
print(gw.state_values[(0,3)])
print(gw.state_values[(1,3)])

0.0
0.0
0.0


**Resultado esperado**
```
V(2,0) = 0.0
V(0,3) = 0.0
V(1,3) = 0.0
```

Cuando el horizonte es $H=1$, el valor del estado $s$ corresponde la recompensa recibida para todas las transiciones a $s'$ posibles + el valor del nuevo estado $s'$:
$$V_{1}^{*}(s)=\max_a \sum_{s'} P(s'|s,a)(R(s,a,s')+\gamma V_{0}^{*}(s'))$$

Pero si en $H=1$ el agente está en un estado terminal, por ejemplo (0,3), el valor de $V_1(0,3)$ se actualizaría así:
$$V_{1}^{*}(0,3)=\max_a \sum_{s'} P(s'|s,a)(R(s,a,s')$$
En este caso se omite el término $V_{0}^{*}(s')$ ya que esta transición no existe al finalizarce el episodio. Por esta razón, el valor de los estados terminales es igual a la recompensa recibida en ellos.




In [56]:
# Se ajusta el valor de los estados terminales
gw.state_values[(0,3)] = 1.0
gw.state_values[(1,3)] = -1.0    

Si el horizonte es $H=2$, para estados no terminales:
$$V_{2}^{*}(s)=\max_a \sum_{s'} P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s'))$$
Por ejemplo, si el agente está en (0,2) y se tiene un descuento $\gamma=0.90$:

* Si pretende ejecutar 'E':
    * Si efectivamente se ejecuta 'E':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.8(0.0+\gamma V_1(0,3))$
    * Si, por ruido, se ejecuta 'N':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.1(0.0+\gamma V_1(0,2))$
    * Si, por ruido, se ejecuta 'S':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.1(0.0+\gamma V_1(1,2))$
    * Valor = $0.8(0.9*1.0)+0.1(0.0)+0.1*(0.0)=0.72$
    
* Si pretende ejecutar 'W':
    * Si efectivamente se ejecuta 'W':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.8(0.0+\gamma V_1(0,1))$
    * Si, por ruido, se ejecuta 'N':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.1(0.0+\gamma V_1(0,2))$
    * Si, por ruido, se ejecuta 'S':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.1(0.0+\gamma V_1(1,2))$
    * Valor = $0.8(0.0)+0.1(0.0)+0.1*(0.0)=0.0$

* Si pretende ejecutar 'N':
    * Si efectivamente se ejecuta 'N':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.8(0.0+\gamma V_1(0,2))$
    * Si, por ruido, se ejecuta 'E':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.1(0.0+\gamma V_1(0,3))$
    * Si, por ruido, se ejecuta 'W':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.1(0.0+\gamma V_1(0,1))$
    * Valor = $0.8(0.0)+0.1(0.9*1.0)+0.1*(0.0)=0.09$
    
* Si pretende ejecutar 'S':
    * Si efectivamente se ejecuta 'S':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.8(0.0+\gamma V_1(1,2))$
    * Si, por ruido, se ejecuta 'E':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.1(0.0+\gamma V_1(0,3))$
    * Si, por ruido, se ejecuta 'W':  $ P(s'|s,a)(R(s,a,s')+\gamma V_{1}^{*}(s')) = 0.1(0.0+\gamma V_1(0,1))$
    * Valor = $0.8(0.0)+0.1(0.9*1.0)+0.1*(0.0)=0.09$
    
Por ende, la acción que maximiza el valor del estado (0,2) es 'E' y $V_{2}^{*}(0,2)=0.72$

### Algoritmo
>Inicializar el valor de todos los estados como $V_{0}^{*}(s)=0$<br>
>Para $k=1, \cdots, H$:<br>
>> Para todos los estados $s$:<br>
>>> $\displaystyle V_{k}^{*}(s)=\max_a \sum_{s'} P(s'|s,a)(R(s,a,s')+\gamma V_{k-1}^{*}(s'))$<br>
>>> $\displaystyle \pi_{k}^{*}(s)=arg\max_a \sum_{s'} P(s'|s,a)(R(s,a,s')+\gamma V_{k-1}^{*}(s'))$<br>

        

Implementemos el algoritmo de value iteration. Las siguientes funciones o atributos pueden ser útiles:
* `gw.get_allowed_actions(state)`: Permite conocer el espacio de acciones.
* `gw.real_actions[action]`: Las posibles acciones (reales) que puede tomar el agente.
* `gw.action_probabilities[action]`: La probabilidad de que el agente tomé las acciones reales.
* `gw.step(state, action, random)`: Permite ejecutar un paso en el gridworld. Con este método se puede conocer el estado siguiente, la recompensa, la accion tomada y la bandera de un estado terminal. Nota: si el parámetro random es Falso, las acciones son determinisiticas (con probabilidad 1, toma la acción dada).
* `gw.state_values[state]`: Los valores del estado *state*.
* `gw.policy[state]`: La acción dada por la política para el estado *state*.

In [57]:
# EDITABLE

def value_iteration(gw, gamma):    
    for state in gw.states:       
        q = dict.fromkeys(gw.get_allowed_actions(state), 0.0) 
        
        # TO DO: Actualice los valores q para Value iteration
        for action in gw.get_allowed_actions(state):
            for i, real_action in enumerate(gw.real_actions[action]):
                state_prima, reward, _, done = gw.step(state, real_action, random = False)
                if done == True:
                    q[action] = reward
                else:
                    q[action] = q[action] + gw.action_probabilities[i]*(reward + gamma*gw.state_values[state_prima])
        
        
        gw.policy[state], gw.state_values[state] =  gw.key_max(q)

**Tenga en cuenta:** Si tiene un error en la función 'value_iteration', despues de corregirlo es necesario que vuelva a ejecutar la celda anterior para tener en cuenta el cambio

Verifique con la siguiente celda que el valor del estado (0,2) es el correcto.

In [58]:
# EDITABLE
value_iteration(gw, gamma)
print(gw.state_values[(0,2)])
print(gw.policy[(0,2)])

0.7200000000000001
E


**Resultado esperado\*:**
```
Para H=2, V(0, 2) = 0.7200000000000001
Para H=2, acción que maximiza el valor = E
```

\* Recuerde reiniciar los valores y ejecutar una única vez el entorno si desea tener un horizonte H=2.

Ahora resuelva el gridworld por *value iteration* utilizando la función que usted implementó:
* Incialice los estados iniciales
* Establezca un $\gamma=0.90$, un horizonte de 15 iteraciones y el estado inicial en (2,0)

In [59]:
# EDITABLE

# Se define una política inicial cualquiera
gw.policy = dict.fromkeys(gw.states, 'N')

# Inicializar valores de los estados
gw.state_values = gw.init_values()

gamma = 0.9
H = 15
init_state = (0,2)

**Visualización:**
1. Ejecute la siguiente celda y haga *clic* sobre la ventana del gridworld. 
2. Podra ver como se actualizan los valores de cada estado presionando la tecla **Espacio**. 
3. Una vez completes las 30 iteraciones, presiona la tecla **Enter** para ver la política
4. Puede cerrar el gridworld presionando la tecla **Esc**.

In [60]:
gw.value_iteration = MethodType(value_iteration, gw)
gw.solve_value_iteration(gamma=gamma, horizon=H, init_state=init_state)

Para un horizonte de 15, los valores de los estados son: 

In [62]:
# EDITABLE
for state in gw.states:
    print(state, gw.state_values[state])

(0, 0) 0.6449688734410074
(0, 1) 0.744380143233867
(0, 2) 0.8477662778685497
(0, 3) 1.0
(1, 0) 0.5663137205171657
(1, 2) 0.5718590329727017
(1, 3) -1.0
(2, 0) 0.49068161767146456
(2, 1) 0.4308405023132072
(2, 2) 0.4754706349982549
(2, 3) 0.27729534324846455


**Resultado esperado:**
```
V(0, 0) = 0.645
V(0, 1) = 0.744
V(0, 2) = 0.848
V(0, 3) = 1.000
V(1, 0) = 0.566
V(1, 2) = 0.572
V(1, 3) = -1.000
V(2, 0) = 0.491
V(2, 1) = 0.431
V(2, 2) = 0.475
V(2, 3) = 0.277
```

La política aprendida es para maximizar la recompensa desde el estado inicial (2,0) es:

In [63]:
# EDITABLE
for state in gw.states:
    print('Acción a tomar en', state, ':', gw.policy[state])

Acción a tomar en (0, 0) : E
Acción a tomar en (0, 1) : E
Acción a tomar en (0, 2) : E
Acción a tomar en (0, 3) : N
Acción a tomar en (1, 0) : N
Acción a tomar en (1, 2) : N
Acción a tomar en (1, 3) : N
Acción a tomar en (2, 0) : N
Acción a tomar en (2, 1) : W
Acción a tomar en (2, 2) : N
Acción a tomar en (2, 3) : W


**Resultado esperado:**
```
Acción a tomar en (0, 0): E
Acción a tomar en (0, 1): E
Acción a tomar en (0, 2): E
Acción a tomar en (0, 3): N
Acción a tomar en (1, 0): N
Acción a tomar en (1, 2): N
Acción a tomar en (1, 3): N
Acción a tomar en (2, 0): N
Acción a tomar en (2, 1): W
Acción a tomar en (2, 2): N
Acción a tomar en (2, 3): W
```

## Policy Iteration

Es otro método que busca aproximar los valores óptimos de cada estado basados en la política actual $\pi(s)$. La politica puede mejorarse con el tiempo, de acuerdo a los valores Q de cada par estado-acción.

### Valores Q
$Q^{*}(s,a)$ es el valor esperado de la utilidad, si el agente comenzara en el estado $s$, tomando la acción $a$ y comportandose óptimamente en adelante.

Los valores Q pueden aproximarse así:
$$Q_{k+1}^{*}(s,a)\leftarrow \sum_{s'} P(s'|s,a)(R(s,a,s')+\gamma \max_{a'}Q_{k}^{*}(s',a'))$$

In [116]:
from types import MethodType
from gridworld import GridWorld, pygame
import copy
from colorama import Fore

gw = GridWorld(3, 4, [(1,1)], [(1,3)], [(0,3)], 0.0)

Se inicializan los valores de los estados en 0.0, al igual que los valores Q para todo par estado acción. Además, inicializamos una política que por defecto siempre toma el norte.

In [117]:
# Inicializar valores y política
import random
gw.state_values = gw.init_values()
gw.state_q_values = gw.init_qvalues()
gw.policy = dict.fromkeys(gw.states, 'N') 
gw.updateGrid()

Verifique la cantidad de valores q:

In [125]:
# EDITABLE
len(gw.state_q_values)*4

44

**Resultado esperado:**

```
num_q_values = 44
```

### Algoritmo

> Inicializar $\pi_{0}(s)$ de forma arbitraria<br>
> Inicializar $V_{0}(s)$ y $Q_{0}(s,a)$ en 0.0 para todos los estados y acciones<br>
> policy_stable = False<br>
> Mientras(not policy_stable):<br>
>> 1. Evaluar la política<br>
>> Para $k=1,\cdots,H$:<br>
>>> Para todos los estados $s$:<br>
>>>> Para todas las acciones $s$ permitidas en $s$:<br>
>>>>> $$Q_{k}(s,a)=\sum_{s'} P(s'|s,\pi(s))(R(s,a,s')+\gamma Q_{k-1}(s',\pi(s')))$$<br>
>>>> Valor del estado $s$ corresponde al valor Q de la acción a ejecutar según la política $\pi_k(s)$ <br>
>>>> $V_{k}(s)=Q_{k}(s,\pi_{k}(s))$ <br>
>
>> 2. Mejorar la política <br>
>> policy_stable = True <br>
>> Para todos los estados $s$:<br>
>>> best_action = $arg\max_{a}Q_k(s,)$<br>
>>> si *best_action* es diferente a lo que dice $\pi_{k}(s)$:<br>
>>>> $\pi_{k}(s) = best\_action$<br>
>>>> policy_stable = False<br>

Implemente la función de *policy evaluation* en donde se actualizan los valores $Q(s,a)$ y $V(s)$ de acuerdo a la política:

Las siguientes funciones pueden ser útiles:

* `gw.get_allowed_actions(state)`: Permite conocer el espacio de acciones.
* `gw.real_actions[action]`: Las posibles acciones (reales) que puede tomar el agente.
* `gw.action_probabilities[action]`: La probabilidad de que el agente tomé las acciones reales.
* `gw.step(state, action, random)`: Permite ejecutar un paso en el gridworld. Con este método se puede conocer el estado siguiente, la recompensa, la accion tomada y la bandera de un estado terminal. Nota: si el parámetro random es Falso, las acciones son determinisiticas (con probabilidad 1, toma la acción dada).
* `gw.state_values[state]`: Los valores del estado *state*.
* `gw.policy[state]`: La acción dada por la política para el estado *state*.

In [119]:
# EDITABLE
def policy_evaluation(gw, gamma):        
    g_tmp = {}
    for state in gw.states:
        q = dict.fromkeys(gw.get_allowed_actions(state), 0.0) 
        # TO DO: Actualice los valores de los estados con Policy Evaluation
        for action in gw.get_allowed_actions(state):
            for i, real_action in enumerate(gw.real_actions[action]):
                state_prima, reward, _, done = gw.step(state, real_action, random = False)
                if done == True:
                    q[action] = reward
                else:
                    q[action] = q[action] + gw.action_probabilities[i]*(reward + gamma*gw.state_q_values[state_prima][gw.policy[state_prima]])
        
            
        #gw.state_values[state] =  gw.max_val(gw.state_q_values[state])
        gw.state_q_values[state] = q.copy()
        gw.state_values[state] = gw.state_q_values[state][gw.policy[state]]

Ahora implemente la función *policy improvement* que permitirá ajustar la política $\pi(s)$ de acuerdo a la nueva estimación de los valores $Q(s,a)$

In [120]:
#EDITABLE
def policy_improvement(gw):
    # Bandera de que la política es estable
    policy_stable = True
    for state in gw.states:
        
        # TO DO: Actualice la política
        best_action, _ =  gw.key_max(gw.state_q_values[state])
        
        if best_action != gw.policy[state]:
            gw.policy[state] = best_action
            policy_stable = False
        
    return policy_stable

Ahora resuelva el gridworld por *policy iteration* utilizando las funciones que usted implementó:
* Incialice los estados iniciales
* Establezca un $\gamma=0.99$, un horizonte de 15 iteraciones y el estado inicial en (2,0)

In [121]:
# EDITABLE
gamma = 0.99
H = 15
init_state = (2,0)

gw.state_values = gw.init_values()
gw.state_q_values = gw.init_qvalues()
gw.policy = dict.fromkeys(gw.states, 'N') 

**Visualización:**
1. Ejecute la siguiente celda y haga clic sobre la ventana del gridworld. 
2. Podrá ir viendo como se actualizan los valores de cada estado cada vez que presionas la tecla **Espacio**. 
3. Tambien puede presionar la tecla Q en la ventana del gridworld para ver los valores Q de cada par (s,a)
4. Si tiene un error en la función 'policy_evaluation' o en 'policy_improvement', despues de corregirlo es necesario volver a correr las 5 celdas anteriores para poder volver a lanzar la interfaz del gridworld.
5. Una vez completadas las iteraciones, aparecerá un mensaje que dice TESTING. Presiona la tecla **Enter** para ver la política
6. Para cerrar el gridworld presiona la tecla **Esc**.

In [122]:
gw.policy_evaluation = MethodType(policy_evaluation, gw)
gw.policy_improvement = MethodType(policy_improvement, gw)
gw.solve_policy_iteration(gamma=gamma, horizon=H, init_state=init_state)

Con *policy iteration*, los valores de los estados son: 

In [123]:
# EDITABLE
for state in gw.states:
    print(state, gw.state_values[state])

(0, 0) 0.9506129584805684
(0, 1) 0.9643218987779846
(0, 2) 0.9766720108876323
(0, 3) 1.0
(1, 0) 0.9387069099493008
(1, 2) 0.8898239753176753
(1, 3) -1.0
(2, 0) 0.9255142363369143
(2, 1) 0.9139166154791812
(2, 2) 0.9010907849586545
(2, 3) 0.7881331049169006


**Resultado esperado:**
```
V(0, 0) = 0.951
V(0, 1) = 0.964
V(0, 2) = 0.977
V(0, 3) = 1.000
V(1, 0) = 0.939
V(1, 2) = 0.890
V(1, 3) = -1.000
V(2, 0) = 0.925
V(2, 1) = 0.913
V(2, 2) = 0.900
V(2, 3) = 0.790
```


La política aprendida es para maximizar la recompensa desde el estado inicial (2,0) es:

In [124]:
# EDITABLE
for state in gw.states:
    print('Acción a tomar en', state, ':', gw.policy[state])

Acción a tomar en (0, 0) : E
Acción a tomar en (0, 1) : E
Acción a tomar en (0, 2) : E
Acción a tomar en (0, 3) : N
Acción a tomar en (1, 0) : N
Acción a tomar en (1, 2) : W
Acción a tomar en (1, 3) : N
Acción a tomar en (2, 0) : N
Acción a tomar en (2, 1) : W
Acción a tomar en (2, 2) : W
Acción a tomar en (2, 3) : S


**Resultado esperado:**
```
Acción a tomar en (0, 0): E
Acción a tomar en (0, 1): E
Acción a tomar en (0, 2): E
Acción a tomar en (0, 3): N
Acción a tomar en (1, 0): N
Acción a tomar en (1, 2): W
Acción a tomar en (1, 3): N
Acción a tomar en (2, 0): N
Acción a tomar en (2, 1): W
Acción a tomar en (2, 2): W
Acción a tomar en (2, 3): S
```

Con esto termina la implementación de los algoritmos de Policy y Value iteration, puede vovlerlos a ejecutar como desee y variar sus parámetros para ver los efectos que estos tienen.